Install dependencies

In [3]:
pip install segment-geospatial

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
pip install leafmap localtileserver

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Import libraries

In [5]:
import os
from samgeo import SamGeo, tms_to_geotiff, get_basemaps
import leafmap

In [6]:
checkpoint = os.path.join("model", "sam_vit_h_4b8939.pth")

In [7]:
sam = SamGeo(
    model_type="vit_h",
    checkpoint=checkpoint,
    sam_kwargs=None,
)

Creating an Interactive Map of HSSU and SLU

In [8]:
mymap = leafmap.Map(center=[38.620186, -90.2745373], zoom=18, height="500px", width = "800px")
mymap.add_basemap("SATELLITE")
mymap

Map(center=[38.620186, -90.2745373], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title…

Let´s create a AOI

In [22]:
if mymap.user_roi_bounds() is not None:
    bbox = mymap.user_roi_bounds()
else:
  bbox = [-90.300, 38.6000, -90.301, 38.6009]

In [15]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
root_dir = "/content/drive/My Drive/"
base_dir = root_dir + 'SAM/'
print("base_dir is: ", base_dir)

Mounted at /content/drive
base_dir is:  /content/drive/My Drive/SAM/


Download maps tiles and mosaic them into a single GeoTIFF file

In [16]:
image = base_dir + "myimage.tif"
mask = base_dir + "myimage_mask.tif"

#os.path.join(base_dir, myname)

In [23]:
tms_to_geotiff(output=image, bbox=bbox, zoom=20, source="Satellite", overwrite=True)

Downloaded image 01/16
Downloaded image 02/16
Downloaded image 03/16
Downloaded image 04/16
Downloaded image 05/16
Downloaded image 06/16
Downloaded image 07/16
Downloaded image 08/16
Downloaded image 09/16
Downloaded image 10/16
Downloaded image 11/16
Downloaded image 12/16
Downloaded image 13/16
Downloaded image 14/16
Downloaded image 15/16
Downloaded image 16/16
Saving GeoTIFF. Please wait...
Image saved to /content/drive/My Drive/SAM/myimage.tif


Display the downloaded image on the map

In [24]:
mymap.layers[-1].visible = False  # turn off the basemap
mymap.add_raster(image, layer_name="SLU")
mymap

Map(bottom=3218012.0, center=[38.60499999999999, -90.30499999999999], controls=(ZoomControl(options=['position…

Segment a geographic image

In [25]:
sam.generate(
    image, mask , batch=True, foreground=True, erosion_kernel=(3, 3), mask_multiplier=255
)

100%|██████████| 4/4 [41:32<00:00, 623.23s/it]


Vectorization of the results

In [28]:
myvector = "SLU.gpkg"
sam.tiff_to_gpkg(mask, myvector, simplify_tolerance=None)

shapefile = base_dir + "SLU.shp"
sam.tiff_to_vector(mask, shapefile)

Visualization of the results

In [27]:
style = {
    "color": "#3388ff",
    "weight": 2,
    "fillColor": "#7c4185",
    "fillOpacity": 0.5,
}
mymap.add_vector(myvector, layer_name="LAL") #, style=style)
mymap

Map(bottom=3218012.0, center=[38.60499999999999, -90.30499999999999], controls=(ZoomControl(options=['position…

The end

For segmenting other type of images

In [ ]:
pip install git+https://github.com/facebookresearch/segment-anything.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/facebookresearch/segment-anything.git to /tmp/pip-req-build-rdkr6qcc
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/segment-anything.git /tmp/pip-req-build-rdkr6qcc
  Resolved https://github.com/facebookresearch/segment-anything.git to commit 6fdee8f2727f4506cfbbe553e23b895e27956588
  Preparing metadata (setup.py) ... done
  Created wheel for segment-anything: filename=segment_anything-1.0-py3-none-any.whl size=36589 sha256=60319b75709f1233b6208bb5ae47d4abcc00b055937347364e6493c5491fe198
  Stored in directory: /tmp/pip-ephem-wheel-cache-absimpyh/wheels/10/cf/59/9ccb2f0a1bcc81d4fbd0e501680b5d088d690c6cfbc02dc99d
Successfully built segment-anything


In [ ]:
! wget https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth

--2023-05-30 00:29:47--  https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 13.35.7.38, 13.35.7.128, 13.35.7.82, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|13.35.7.38|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2564550879 (2.4G) [binary/octet-stream]
Saving to: ‘sam_vit_h_4b8939.pth’

sam_vit_h_4b8939.pt 100%[===================>]   2.39G   110MB/s    in 25s     

2023-05-30 00:30:12 (97.9 MB/s) - ‘sam_vit_h_4b8939.pth’ saved [2564550879/2564550879]



In [ ]:
model_type="vit_h"
samcheckpoint = "sam_vit_h_4b8939.pth"

In [ ]:
image = base_dir + "casa.jpg"

In [ ]:
from segment_anything import SamAutomaticMaskGenerator, sam_model_registry
sam = sam_model_registry[model_type](checkpoint=samcheckpoint)
mask_generator = SamAutomaticMaskGenerator(sam)
masks = mask_generator.generate(image)

AttributeError: ignored